In [1]:
from numpy.random import randint
from numpy.random import rand

In [2]:
def objective(x):
	return x[0]**2.0 + x[1]**2.0

In [3]:
def decode(bounds, n_bits, bitstring):
	decoded = list()
	largest = 2**n_bits
	for i in range(len(bounds)):
		# extract the substring
		start, end = i * n_bits, (i * n_bits)+n_bits
		substring = bitstring[start:end]
		# convert bitstring to a string of chars
		chars = ''.join([str(s) for s in substring])
		# convert string to integer
		integer = int(chars, 2)
		# scale integer to desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
		# store
		decoded.append(value)
	return decoded

In [4]:
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]

In [5]:
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

In [6]:
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

In [9]:
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
	# enumerate generations
	for gen in range(n_iter):
		# decode population
		decoded = [decode(bounds, n_bits, p) for p in pop]
		# evaluate all candidates in the population
		scores = [objective(d) for d in decoded]
		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print(">iteration %d, new best (%s) = %f" % (gen,  decoded[i], scores[i]))
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			for c in crossover(p1, p2, r_cross):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children
	return [best, best_eval]

In [10]:
bounds = [[-5.0, 5.0], [-5.0, 5.0]]
# define the total iterations
n_iter = 100
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds)) #0.03125
# perform the genetic algorithm search
print(f'Starting genetic algorithm\n')
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
decoded = decode(bounds, n_bits, best)
print(f'\nGenetic algorithm completed\n')
print(f'Best solution: {decoded}')
print(f'Fitness score of the best solution: {score:.5f}')

Starting genetic algorithm

>iteration 0, new best ([0.869598388671875, -1.7596435546875]) = 3.852547
>iteration 0, new best ([-1.92047119140625, 0.064849853515625]) = 3.692415
>iteration 0, new best ([-0.301361083984375, -1.735687255859375]) = 3.103429
>iteration 0, new best ([1.632843017578125, -0.437469482421875]) = 2.857556
>iteration 0, new best ([-0.59600830078125, 0.048980712890625]) = 0.357625
>iteration 1, new best ([-0.09063720703125, -0.091705322265625]) = 0.016625
>iteration 2, new best ([-0.09063720703125, -0.081939697265625]) = 0.014929
>iteration 3, new best ([-0.07843017578125, 0.039215087890625]) = 0.007689
>iteration 4, new best ([0.02899169921875, 0.074310302734375]) = 0.006363
>iteration 5, new best ([0.02899169921875, 0.054473876953125]) = 0.003808
>iteration 6, new best ([-0.01312255859375, -0.01678466796875]) = 0.000454
>iteration 8, new best ([-0.013580322265625, -0.00701904296875]) = 0.000234
>iteration 13, new best ([-0.00823974609375, -0.01190185546875]) = 0.